# Resume Screening

Henry Yost, Riya Ashok, Angelina Jordan, Gokul Giridharan, and Refugio Zepeda Jr

## 1. What Data do we have?
> Answer

## 2. What we would like to know:
> Answer more in depth

Given a dataset of resumes, can we assess if someone is qualified for the job position?

## 3. Importing Dataset & Cleaning

In [50]:
import pandas as pd

df_orig = pd.read_csv("Resume.csv")
df = df_orig.drop("Resume_html", axis=1) # Drop HTML, because we do not need it

# All possible categories: HR, Designer, Information-Technology, Teacher, Advocate, Business-Development, 
# Healthcare, Fitness, Agriculture, BPO, Sales, Consultant, Digital-Media, Automobile, Chef, 
# Finance, Apparel, Engineering, Accountant, Construction, Public-Relations, Banking, Arts, Aviation

# Remove non-relevant data noise (keep IT, Healthcare, Sales, Engineering, HR)
df = df[df["Category"].isin(["INFORMATION-TECHNOLOGY", "HEALTHCARE", "SALES", "ENGINEERING", "HR"])].reset_index(drop=True)

df.to_csv("new_data.csv")

In [68]:
# Using ReGeX to clean the Resume strings
import re

def clean_resume(text):
    text = text.lower() # Make everything lowercase.
    text = re.sub(r'\s+', ' ', text) # replaces whitespace characters with a single space.
    text = re.sub(r'\/+', ' ', text) # replaces / characters with a single space (some resumes use backslashes)
    text = re.sub(r'[^a-z0-9\s\.\-]', '', text) # keeps letters, numbers, whitespace, periods, and dashes.
    text = re.sub(r'\s+', ' ', text).strip() # replaces leading and trailing whitespace characters.
    return text

df["Cleaned_str"] = df["Resume_str"].apply(clean_resume)

# Also, there are no NaN values, so we don't need to drop any rows.

df.head()

,ID,Resume_str,Category,Cleaned_str
0,16852973,HR ADMINISTRATOR/MARKETING ASSOCIATE\...,HR,hr administrator marketing associate hr admini...
1,22323967,"HR SPECIALIST, US HR OPERATIONS ...",HR,hr specialist us hr operations summary versati...
2,33176873,HR DIRECTOR Summary Over 2...,HR,hr director summary over 20 years experience i...
3,27018550,HR SPECIALIST Summary Dedica...,HR,hr specialist summary dedicated driven and dyn...
4,17812897,HR MANAGER Skill Highlights ...,HR,hr manager skill highlights hr skills hr depar...


## 4. Exploratory Data Analysis (EDA)